In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle
from tqdm import tqdm

from amelia_datatools.utils.common import OUTPUT_DIR, DATA_DIR, VERSION

out_dir = f"{OUTPUT_DIR}/count"
os.makedirs(out_dir, exist_ok=True)


In [ ]:
airport = "kbos"
base_dir = DATA_DIR
traj_version = VERSION

traj_dir = os.path.join(base_dir, f'traj_data_{traj_version}', 'raw_trajectories', f'{airport}')

traj_files = [os.path.join(traj_dir, f) for f in os.listdir(traj_dir)]
print(traj_files[0])

data = pd.read_csv(traj_files[0])
data.head()

In [ ]:
agent_counts = {
    'total': [], '0': [], '1': [], '2': []
}
total_count = 0
for f in tqdm(traj_files):
    data = pd.read_csv(f)

    unique_frames = np.unique(data.Frame)
    for frame in unique_frames:
        agents = data[data.Frame == frame]

        agent_counts['total'].append(agents.shape[0])
        agent_counts['0'].append(np.where(agents.Type == 0.0)[0].shape[0])
        agent_counts['1'].append(np.where(agents.Type == 1.0)[0].shape[0])
        agent_counts['2'].append(np.where(agents.Type == 2.0)[0].shape[0])


In [ ]:
agent_count_stats = {}
for k, v in agent_counts.items():
    v = np.asarray(v)
    agent_count_stats[k] = {
        "min": round(v.min().astype(float), 5),
        "max": round(v.max().astype(float), 5),
        "mean": round(v.mean().astype(float), 5),
        "std": round(v.std().astype(float), 5)
    }

    agent_counts[k] = v

agent_count_stats

In [ ]:
with open(f'{out_dir}/{airport}_stats.json', 'w') as f:
    json.dump(agent_count_stats, f, indent=2)

with open(f'{out_dir}/{airport}.pkl', 'wb') as handle:
    pickle.dump(agent_counts, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
name = {'total': 'Total', '0': 'Aircraft', '1': 'Vehicle', '2': 'Unknown'}
colors = {'ksea': 'blue', 'kewr': 'red', 'kbos': 'green', 'kmdw': 'orange'}
for key, value in name.items():
    arr = agent_counts[key]

    plt.hist(arr, bins = int(arr.max()), color=colors[airport])
    plt.title(f"{value}")
    plt.xlabel("Number of Agents per Timestep")
    plt.ylabel("Count")
    plt.savefig(f"{out_dir}/{airport}_{key}.png", dpi=600)
    plt.close()

In [ ]:
for key, value in name.items():
    arr = agent_counts[key]
    q_lower = np.quantile(arr, 0.05)
    q_upper = np.quantile(arr, 0.95)

    arr = arr[(arr >= q_lower) & (arr <= q_upper)]

    plt.hist(arr, bins = int(arr.max()), color=colors[airport])
    plt.title(f"{value}")
    plt.xlabel("Number of Agents per Timestep")
    plt.ylabel("Count")
    plt.savefig(f"{out_dir}/{airport}_{key}_iqr.png", dpi=600)
    plt.close()